In [2]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D , MaxPool2D , Flatten , Dense , Dropout
import cv2
import os

pd.set_option('display.max_rows',None)                     # to display max rows and column
pd.set_option('display.max_columns',None)

In [2]:


image_size = 200  #  you can keep any image size..but image size plays big role in model creation
                  # if image size is less..computation needed is less..but at the same time model performance may be bad
path1 = r'E:\PGA $$ 23 $$\Deep Learning\CNN\EMOTION'
cate  = ['ANGRY','HAPPY']
    
input_image = []    
for i in cate:
    folders = os.path.join(path1 ,i)
    label   = cate.index(i)   # we need to tell software which image is of cat and which is of dog
    for image in os.listdir(folders):
        image_path  = os.path.join(folders , image)
        image_array =  cv2.imread(image_path) #  using the cv2 i am reading the image and storing in variable image_array
        image_array = cv2.resize(image_array , (image_size  , image_size) ) # resizing each image to 200 * 200
        input_image.append([image_array , label])

In [3]:
len(input_image)

11025

In [4]:
np.random.shuffle(input_image)


X  = []
Y =  []

for X_values , labels in input_image:
    X.append(X_values)
    Y.append(labels)


X = np.array(X)
Y = np.array(Y)

X_train = X[:10000]
X_test = X[10000:]

Y_train = Y[:10000]
Y_test = Y[10000:]

X_train = X_train/255
X_test = X_test/255
X_train.shape


(10000, 200, 200, 3)

In [5]:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPool2D , Flatten , Dense , Dropout

model = Sequential()
model.add(Conv2D(filters = 16 , kernel_size =(5,5) , activation ='relu'     ))
#model.add(Conv2D(filters = 20 , kernel_size =(3,3) , activation ='relu'     ))

model.add(MaxPool2D(pool_size =(2,2)))



model.add(Flatten())
model.add(Dense(128 , activation = 'relu' , input_shape = X.shape[1:]))

model.add(Dense(2 , activation= 'softmax'))

model.compile(optimizer='adam',   loss = 'sparse_categorical_crossentropy'  , metrics = ['accuracy'] )


In [6]:
model.fit(X_train , Y_train, epochs = 2,validation_split =.2  )

Epoch 1/2
250/250 [==============================] - 562s 2s/step - loss: 0.6627 - accuracy: 0.9115 - val_loss: 0.0964 - val_accuracy: 0.9765
Epoch 2/2
250/250 [==============================] - 195s 781ms/step - loss: 0.0977 - accuracy: 0.9843 - val_loss: 0.0741 - val_accuracy: 0.9835


In [7]:
pred1 = model.predict(X_test)
pred1

array([[3.5703611e-02, 9.6429640e-01],
       [9.9933124e-01, 6.6875300e-04],
       [9.9991488e-01, 8.5101557e-05],
       ...,
       [9.9963689e-01, 3.6306499e-04],
       [9.9982268e-01, 1.7731873e-04],
       [5.5938433e-03, 9.9440610e-01]], dtype=float32)

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(pred1,1),Y_test)

0.9853658536585366

In [9]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.argmax(pred1,1),Y_test)

array([[451,   6],
       [  9, 559]], dtype=int64)

In [ ]:
model.save('mask_&_no_mask.h5')   # Save after model fit

In [18]:
import cv2 
from PIL import Image
import numpy as np
from keras.models import load_model
#model = load_model("F:\pranjal\knowledge\data analytics\R programing\conducting classes\PythonModel\my_mask_NoMask_model.h5") 
#model=load_model(r"C:\Users\ASHISH SHINDE\Inclass\mask_no_mask.h5")

results={1:'ANGRY',0:'HAPPY'}
GR_dict={1:(0,0,255),0:(0,255,255)}

rect_size = 4
cap = cv2.VideoCapture(0)


haarcascade = cv2.CascadeClassifier(r"E:\PGA $$ 23 $$\Deep Learning\CNN\Harcascade\haarcascade_frontalface_default.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1)
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(200,200))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,200,200,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    cv2.imshow('Liv Camera',   im)
    key = cv2.waitKey(10)
    
    if key == 27: # use the escape key
        break

cap.release()

cv2.destroyAllWindows()

In [18]:
model.save('EMOTION.h5')

In [1]:
from keras.models import load_model
#model = load_model("F:\pranjal\knowledge\data analytics\R programing\conducting classes\PythonModel\my_mask_NoMask_model.h5") 
model=load_model(r"C:\Users\ASHISH SHINDE\Inclass\EMOTION.h5")

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
#model = load_model("F:\pranjal\knowledge\data analytics\R programing\conducting classes\PythonModel\my_mask_NoMask_model.h5") 
#model=load_model(r"C:\Users\ASHISH SHINDE\Inclass\mask_no_mask.h5")

results={1:'Angry',0:'Happy'}
GR_dict={1:(0,0,255),0:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier(r"E:\PGA $$ 23 $$\Deep Learning\CNN\Harcascade\haarcascade_frontalface_default.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1)
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(200,200))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,200,200,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    cv2.imshow('Liv Camera',   im)
    key = cv2.waitKey(10)
    
    if key == 27: # use the escape key
        break

cap.release()

cv2.destroyAllWindows()

